In [3]:
!pip install mediapipe==0.10.14 --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 33.4 MB/s eta 0:00:00
  Attempting uninstall: mediapipe
    Found existing installation: mediapipe 0.10.21
    Uninstalling mediapipe-0.10.21:
      Successfully uninstalled mediapipe-0.10.21


In [10]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

# Load overlay image (must be RGBA)
cat_overlay = cv2.imread('catEars.png', cv2.IMREAD_UNCHANGED)

# Helper: Overlay RGBA with alpha blending
def overlay_alpha(img, overlay, x, y):
    h, w = overlay.shape[:2]
    if x < 0 or y < 0 or x + w > img.shape[1] or y + h > img.shape[0]:
        return img  # skip if out of bounds
    alpha = overlay[:, :, 3] / 255.0
    for c in range(3):
        img[y:y + h, x:x + w, c] = (1 - alpha) * img[y:y + h, x:x + w, c] + alpha * overlay[:, :, c]
    return img

# Improved: rotate image with padding to avoid cropping
def rotate_image(image, angle_deg):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Rotation matrix
    M = cv2.getRotationMatrix2D(center, angle_deg, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # New image size
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # Adjust matrix for translation
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]

    # Rotate and return
    return cv2.warpAffine(image, M, (new_w, new_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# Load video
cap = cv2.VideoCapture('/content/input.mp4')  # Change this path as needed
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
cap.release()

# Process each frame
output_frames = []
for frame in frames:
    h, w = frame.shape[:2]
    result = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    if not result.multi_face_landmarks:
        output_frames.append(frame)
        continue

    lm = result.multi_face_landmarks[0].landmark
    p1 = np.array([lm[127].x * w, lm[127].y * h])  # left forehead
    p2 = np.array([lm[356].x * w, lm[356].y * h])  # right forehead
    p_center = np.array([lm[1].x * w, lm[1].y * h])  # nose tip

    face_width = np.linalg.norm(p1 - p2)
    ear_width = int(face_width * 1.2)
    ear_height = int(ear_width * cat_overlay.shape[0] / cat_overlay.shape[1])

    # Resize filter to match face width
    resized_overlay = cv2.resize(cat_overlay, (ear_width, ear_height), interpolation=cv2.INTER_AREA)

    # Compute face tilt angle (in degrees), and flip it!
    angle_rad = np.arctan2(p2[1] - p1[1], p2[0] - p1[0])
    angle_deg = np.degrees(angle_rad)

    # Rotate overlay to match head orientation
    rotated_overlay = rotate_image(resized_overlay, -angle_deg)  # ← FIX: negative for correct tilt

    # Position: align center x and slightly above nose
    top_x = int(p_center[0] - rotated_overlay.shape[1] // 2)
    top_y = int(p_center[1] - rotated_overlay.shape[0] * 0.8)  # ← FIX: adjust for nose alignment

    out = overlay_alpha(frame.copy(), rotated_overlay, top_x, top_y)
    output_frames.append(out)

# Save output as MP4
bgr_frames = [cv2.cvtColor(f, cv2.COLOR_RGB2BGR) for f in output_frames]
height, width = bgr_frames[0].shape[:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('cat_filter_output.mp4', fourcc, 10.0, (width, height))

for frame in bgr_frames:
    out.write(frame)
out.release()

print("✅ Saved: cat_filter_output.mp4")


✅ Saved: cat_filter_output.mp4
